<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Importing necessary libraries
Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import pandas as pd
import numpy as np
import urllib.request #library to fetch the HTML from the URL
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans # import k-means from clustering stage

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

try:
    from bs4 import BeautifulSoup #library to scrapping information 
except ImportError:
    !conda install -c anaconda beautifulsoup4 --yes
try:
    from geopy.geocoders import Nominatim
except:
    !conda install -c conda-forge geopy --yes
try:
    import folium # map rendering library
except:
    !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab


## Scrapping Wikipedia page

We specify the URL of the Wikipedia page. Using urllib.request library, we want to query the page and put HTML data into a variable.

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "html.parser")

We retrive all instances of the \<table\> tag. We choose this one which contains List of postal codes of Canada "wikitable sortable". We loop over HTML tags and assigns values to each of list (A - PostalCode , B - Borough , C - Neighborhood)

In [4]:
all_tables = soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
postalCode = []
borough = []
neighborhood = []

for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        postalCode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))

## Create dataframe from scrapped data

We create DataFrame and assign scrapped data to proper columns. We clean our data.

In [5]:
df = pd.DataFrame(postalCode,columns = ['PostalCode'])
df['Borough'] = borough
df['Neighborhood'] = neighborhood
df = df.replace('\n','', regex=True)
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)

if df[df.Neighborhood=='Not assigned'].shape[0] != 0:
    df.loc[df['Neighborhood']=='Not assigned', 'Neighborhood'] = df['Borough']
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let's check size of created DataFrame

In [6]:
df.shape

(103, 3)

Now let's add coordinates for each Postal Code to our DataFrame

In [7]:
df_coords = pd.read_csv("Geospatial_Coordinates.csv")
df_coords.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df = pd.merge(df, df_coords, on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Explore Neighborhoods in Toronto
We select only rows where Borugh contains 'Toronto'

In [8]:
df_toronto = df[df['Borough'].str.contains("Toronto")]
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Let's get the geographical coordinates of Central Toronto

In [9]:
address = 'Central Toronto, Toronto'
geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Downtown Toronto are 43.6534817, -79.3839347.


Let's visualize Downtown Toronto the neighborhoods in it. 

In [10]:
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

### Define Foursquare Credentials and Version

In [11]:
CLIENT_ID = 'NRZ1O5P2RYDGKNGVRIOVIFHSEXQ4UOOQYEHAH55H3RAAU3BP' # your Foursquare ID
CLIENT_SECRET = '2ULQYKF1B2EVW2K532CZHZLYPZNHRFZTBN30VZZV3IWRPJFH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NRZ1O5P2RYDGKNGVRIOVIFHSEXQ4UOOQYEHAH55H3RAAU3BP
CLIENT_SECRET:2ULQYKF1B2EVW2K532CZHZLYPZNHRFZTBN30VZZV3IWRPJFH


## Get top 100 venues from all Neighborhoods in Downtown,West,East and Central Toronto
Function to get venues for all the Neighborhoods

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']#[1]['venue']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Create new dataframe with Toronto venues

In [13]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [14]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,66,66,66,66,66,66
Christie,16,16,16,16,16,16
Church and Wellesley,72,72,72,72,72,72
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


Let's find out how many unique categories can be curated from all returned venues

In [15]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


## Analyze each neighborhood

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#change name of column Neighborhood, becuase in this case it's Venue Category
toronto_onehot.rename(columns={'Neighborhood':'Neighborhood_venue'}, inplace=True)

# add neighborhood column to dataframe
toronto_onehot.insert(0,"Neighborhood",toronto_venues['Neighborhood'],True) 
toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Let's group rows by neighborhood

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017544,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.015152,0.0,0.0,0.015152,0.0,0.015152


### Print each neighborhood along with the top 3 most common venues

In [47]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2      Pharmacy  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.12
1             Brewery  0.06
2       Auto Workshop  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.06
2                Café  0.05


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12


----Church and Wellesley----
                 venue  freq
0

## Let's put that into DataFrame
Let's write a function to sort the venues in descending order

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe with top 6 venues for each neighborhood

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Beer Bar,Cheese Shop,Pharmacy,Farmers Market,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Pet Store,Furniture / Home Store,Convenience Store,Stadium,Restaurant,Italian Restaurant,Intersection
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Auto Workshop,Brewery,Burrito Place,Farmers Market,Spa,Fast Food Restaurant,Restaurant,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Plane,Bar,Coffee Shop,Boat or Ferry,Boutique,Sculpture Garden
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Department Store,Salad Place


# Cluster Neighborhoods
Run k-means to cluster into 5 clusters

In [42]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

Let's create a new DataFrame with Cluster label

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Theater,Ice Cream Shop,French Restaurant,Health Food Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Diner,College Cafeteria,Yoga Studio,Sandwich Place,Café,Smoothie Shop,Mexican Restaurant,Italian Restaurant,Beer Bar
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Italian Restaurant,Hotel,Bubble Tea Shop,Café,Japanese Restaurant,Cosmetics Shop,Tea Room,Ramen Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Restaurant,Café,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Cocktail Bar,Creperie,Breakfast Spot,Beer Bar
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Neighborhood_venue,Health Food Store,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


### Let's visualize the resulting cluster

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

#### Cluster 1

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Park,Breakfast Spot,Pub,Café,Theater,Ice Cream Shop,French Restaurant,Health Food Store
4,Downtown Toronto,0,Coffee Shop,Diner,College Cafeteria,Yoga Studio,Sandwich Place,Café,Smoothie Shop,Mexican Restaurant,Italian Restaurant,Beer Bar
9,Downtown Toronto,0,Clothing Store,Coffee Shop,Italian Restaurant,Hotel,Bubble Tea Shop,Café,Japanese Restaurant,Cosmetics Shop,Tea Room,Ramen Restaurant
15,Downtown Toronto,0,Restaurant,Café,Coffee Shop,Clothing Store,Cosmetics Shop,American Restaurant,Cocktail Bar,Creperie,Breakfast Spot,Beer Bar
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Beer Bar,Cheese Shop,Pharmacy,Farmers Market,Seafood Restaurant
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Burger Joint,Bubble Tea Shop,Bar,Department Store,Salad Place
30,Downtown Toronto,0,Coffee Shop,Café,Clothing Store,Hotel,Restaurant,Deli / Bodega,Thai Restaurant,Gym,Salad Place,Sushi Restaurant
36,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Restaurant,Sporting Goods Shop,Brewery,Fried Chicken Joint,Plaza
37,West Toronto,0,Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Men's Store,Café,Coffee Shop,Vietnamese Restaurant,Restaurant,Juice Bar,Record Shop
41,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Bubble Tea Shop,Spa,Bookstore


#### Cluster 2

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,1,Park,Bus Line,Swim School,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
68,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
91,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 3

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,2,Garden,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,3,Pub,Neighborhood_venue,Health Food Store,Trail,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
25,Downtown Toronto,3,Grocery Store,Café,Park,Italian Restaurant,Restaurant,Baby Store,Candy Store,Coffee Shop,Diner,Nightclub
31,West Toronto,3,Bakery,Pharmacy,Music Venue,Café,Middle Eastern Restaurant,Supermarket,Recording Studio,Bar,Bank,Pet Store
43,West Toronto,3,Café,Breakfast Spot,Coffee Shop,Pet Store,Furniture / Home Store,Convenience Store,Stadium,Restaurant,Italian Restaurant,Intersection
54,East Toronto,3,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Pet Store,Park,Neighborhood_venue,Food
69,West Toronto,3,Café,Mexican Restaurant,Thai Restaurant,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant,Speakeasy,Bookstore,Convenience Store,Grocery Store
74,Central Toronto,3,Sandwich Place,Café,Coffee Shop,Pub,History Museum,Burger Joint,Donut Shop,Middle Eastern Restaurant,Indian Restaurant,BBQ Joint
80,Downtown Toronto,3,Café,Sandwich Place,Bar,Japanese Restaurant,Bookstore,Restaurant,Bakery,Beer Bar,Beer Store,Italian Restaurant
81,West Toronto,3,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Pub,Bookstore,Tea Room,Yoga Studio,Falafel Restaurant,Indie Movie Theater


#### Cluster 5

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,4,Playground,Summer Camp,Tennis Court,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


First try:  4 Clusters with 6 top venues per 1 Neighborhood didn't give satisfied outcome, next tries were better...

Finally I chose 5 Cluster with 10 top venues per 10 Neighborhood. We can distinguish two bigger clusters (1 and 4).
 - Cluster 1 - top venues are coffe shops, bars, restaurannt 
 - Cluster 2 - top venues are parks and other things for public use
 - Cluster 3 - has nothing special, could be merged with another cluster
 - Cluster 4 - similarily to cluster 1
 - Cluster 5 - could merged with cluster 2 

Cluster 2 and 5 can be for people who like active style of life. They mostly are in Central Toronto.
Cluster 1 and 4 are good for people who like to eat, entertainment. 
I think cluster 3 is compromise between entertainment and healthy life style.